In [9]:
using CSV, DataFrames, Statistics, Random, GLMNet
using DataFrames: Not

# load data
df = CSV.read("parkinsons_updrs.csv", DataFrame)

println(describe(df, :min, :max, :median, :nmissing))

if "subject#" in names(df)
    rename!(df, "subject#" => "subject")
end

println("Columns loaded:")
println(names(df))

# extract target and features
target = "total_UPDRS"
remove_cols = ["subject", "motor_UPDRS", "total_UPDRS"]
features = filter(col -> !(col in remove_cols), names(df))

println("\nUsing $(length(features)) features:")
println(features)

# standardize features
X_raw = Matrix(df[:, features])
X_mean = mean(X_raw, dims=1)
X_std  = std(X_raw, dims=1)
X_std[X_std .== 0] .= 1.0

X = (X_raw .- X_mean) ./ X_std
y = df[:, target]

println("\nStandardization complete.")
println("X size = ", size(X), ", y length = ", length(y))


# patient-level 5-fold cross-validation
subjects = unique(df[:, "subject"])
K = 5
Random.seed!(15095)

shuffled = Random.shuffle(subjects)
folds = [shuffled[i:K:length(shuffled)] for i in 1:K]

println("\nPatient-level folds:")
println(folds)


# GLMNet Lasso training function
function evaluate_fold(train_idx, test_idx)
    X_train, y_train = X[train_idx, :], y[train_idx]
    X_test,  y_test  = X[test_idx, :], y[test_idx]

    # Cross-validated LASSO (α = 1)
    cv = glmnetcv(X_train, y_train, alpha=1.0)

    # Index of best λ (min CV error)
    best_idx = argmin(cv.meanloss)
    λ_best = cv.lambda[best_idx]

    # Fit LASSO path
    fit = glmnet(X_train, y_train, alpha=1.0)

    # Predict using best index (old GLMNet API)
    y_pred = predict(fit, X_test, best_idx)
    y_pred = vec(y_pred)

    fold_mae  = mean(abs.(y_test - y_pred))
    fold_rmse = sqrt(mean((y_test - y_pred).^2))

    return fold_mae, fold_rmse, λ_best
end


# 5-Fold Patient-Level LASSO Cross-Validation
mae_list = Float64[]
rmse_list = Float64[]
lambda_list = Float64[]

println("\n======================================")
println(" Running 5-Fold Patient-Level LASSO ")
println("======================================")

for fold_num in 1:K
    test_subjects = folds[fold_num]
    train_subjects = setdiff(subjects, test_subjects)

    train_idx = findall(in(train_subjects), df[:, "subject"])
    test_idx  = findall(in(test_subjects), df[:, "subject"])

    println("\n--- Fold $fold_num / $K ---")

    fold_mae, fold_rmse, λ_best = evaluate_fold(train_idx, test_idx)

    println("λ_best = ", λ_best)
    println("Fold MAE  = ", fold_mae)
    println("Fold RMSE = ", fold_rmse)

    push!(mae_list, fold_mae)
    push!(rmse_list, fold_rmse)
    push!(lambda_list, λ_best)
end

# summary
println("\n========== FINAL LASSO RESULTS ==========")
println("Mean MAE:   ", round(mean(mae_list), digits=4))
println("Mean RMSE:  ", round(mean(rmse_list), digits=4))
println("Avg λ_best: ", round(mean(lambda_list), digits=6))
println("=========================================\n")

22×5 DataFrame
 Row │ variable       min        max           median     nmissing
     │ Symbol         Real       Real          Float64    Int64
─────┼─────────────────────────────────────────────────────────────
   1 │ subject#        1          42           22.0              0
   2 │ age            36          85           65.0              0
   3 │ sex             0           1            0.0              0
   4 │ test_time      -4.2625    215.49        91.523            0
   5 │ motor_UPDRS     5.0377     39.511       20.871            0
   6 │ total_UPDRS     7.0        54.992       27.576            0
   7 │ Jitter(%)       0.00083     0.09999      0.0049           0
   8 │ Jitter(Abs)     2.25e-6     0.00044559   3.453e-5         0
   9 │ Jitter:RAP      0.00033     0.05754      0.00225          0
  10 │ Jitter:PPQ5     0.00043     0.06956      0.00249          0
  11 │ Jitter:DDP      0.00098     0.17263      0.00675          0
  12 │ Shimmer         0.00306     0.26863      0